Here I do cleaning of Netflix dataset

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('C:/Users/nikit/data_analysis/netflix/datasets')

with open('netflix1.csv','r', encoding='utf8') as file:
    netflix = pd.read_csv(file, index_col='show_id')

Some funcs to make my life easier

In [2]:
def get_null(df: pd.DataFrame) -> pd.DataFrame:
    null_mask = df.isin(['Not Given']).any(axis=1)
    null_frame = df[null_mask]
    df_not_null = df[null_mask == False]
    
    return null_frame, df_not_null

def print_missing(df: pd.DataFrame) -> None:
    print('Missing values:')
    print(df.isin(['Not Given']).sum())
    print('Total missing values:', df.isin(['Not Given']).sum().sum())

Let's see how many rows are missing

In [3]:
print('Initial nulls:')
print_missing(netflix)

Initial nulls:
Missing values:
type               0
title              0
director        2588
country          287
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
dtype: int64
Total missing values: 2875


That's around 30% so I can't drop them right now. I'll try to treat them. 

- As first step, I will look for similarities in titles (in first two words)
- Then, I will treat missing countries firtsly by similarities in type, director, and rating and secondly by director only

In [5]:
null_frame, netflix_not_null = get_null(netflix)
split_title = netflix_not_null['title'].str.replace(r'(\S+)\s(\S+).*', r'\1 \2', regex=True)
split_title = pd.DataFrame(split_title, columns=['title'])
split_title.index.name = 'show_id'

We got first two words of each title in dataset

In [6]:
for id in null_frame.index.tolist():
    row = null_frame.loc[id]
    title = row['title'].split(' ')[:2]
    title = ' '.join(title)
    rep_id = split_title[split_title['title'] == title]
    if not rep_id.empty:
        rep_id = rep_id.index[0]
        rep_row = netflix_not_null.loc[rep_id]
        if row['director'] == 'Not Given':
            netflix.loc[id, 'director'] = rep_row['director']
        if row['country'] == 'Not Given':
            netflix.loc[id, 'country'] = rep_row['country']

In [7]:
print('After treatment by similarities in titles:')
print_missing(netflix)

After treatment by similarities in titles:
Missing values:
type               0
title              0
director        2301
country          235
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
dtype: int64
Total missing values: 2536


We were able to treat about 300 missing directors and 50 missing countries

In [14]:
null_frame, netflix_not_null = get_null(netflix)
type_dir = netflix_not_null[['type', 'director', 'country']]
# Treatment by other similarities
for id in null_frame.index.tolist():
    row = null_frame.loc[id]
    if row['country'] == 'Not Given':
        similar = type_dir[(type_dir['type'] == row['type'])&
                            (type_dir['director'] == row['director'])]
        if not similar.empty:
            suggest = similar['country'].value_counts().idxmax()
            netflix.loc[id, 'country'] = suggest

print('After treatment by other similarities:')
print_missing(netflix)

After treatment by other similarities:
Missing values:
type               0
title              0
director        2301
country          234
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
dtype: int64
Total missing values: 2535


It's not doing much now, bc those directors seem to have only one commitment in this dataset. So I'm not really able to do much about that without creating a ton of misleading data 

In [15]:
null_frame, netflix_not_null = get_null(netflix)

# Treatment by director
for id in null_frame.index.tolist():
    row = null_frame.loc[id]
    if row['country'] == 'Not Given':
        similar = netflix_not_null[netflix_not_null['director'] == row['director']]
        if not similar.empty:
            suggest = similar['country'].value_counts().idxmax()
            netflix.loc[id, 'country'] = suggest

print('After treatment by director:')
print_missing(netflix)

After treatment by director:
Missing values:
type               0
title              0
director        2301
country          234
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
dtype: int64
Total missing values: 2535


Still nothing

As last attempt I'll try to group data by date_added

In [17]:
for group, frame in netflix.groupby('date_added'):
    null_frame, netflix_not_null = get_null(frame)
    for id in null_frame.index.tolist():
        row = null_frame.loc[id]
        if row['director'] == 'Not Given':
            similar = netflix_not_null[netflix_not_null['country'] == row['country']]
            if not similar.empty:
                suggest = similar['director'].value_counts().idxmax()
                netflix.loc[id, 'director'] = suggest

print('After treatment by date_added:')
print_missing(netflix)

After treatment by date_added:
Missing values:
type               0
title              0
director        1457
country          234
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
dtype: int64
Total missing values: 1691


Fantastically! Let's check if there are weird values (e.g. directors with 100+ movies)

In [19]:
print(netflix['director'].value_counts())

Not Given                           1457
Charlie Haskell, Koichi Sakamoto      34
Rajiv Chilaka                         21
David Briggs                          20
Alastair Fothergill                   19
                                    ... 
Sandeep Reddy Vanga                    1
Randall Lobb                           1
Teng Huatao                            1
Nzingha Stewart                        1
Mozez Singh                            1
Name: director, Length: 4528, dtype: int64


Charlie Haskell and his friend are likely filmed all stuff about Power Rangers, check if there is evidence in original dataset

In [21]:
with open('netflix1.csv','r', encoding='utf8') as file:
    _netflix = pd.read_csv(file, index_col='show_id')
    print(_netflix.query('director == "Charlie Haskell, Koichi Sakamoto"')['title'])

show_id
s7779    Power Rangers Super Megaforce: The Legendary B...
Name: title, dtype: object


Okay, it's all right. Let's finally check Rajiv Chilaka, his major work is Chhota Beem

In [22]:
with open('netflix1.csv','r', encoding='utf8') as file:
    _netflix = pd.read_csv(file, index_col='show_id')
    print(_netflix.query('director == "Rajiv Chilaka"')['title'])

show_id
s420                         Chhota Bheem: Bheem vs Aliens
s407                          Chhota Bheem - Neeli Pahaadi
s408                                 Chhota Bheem & Ganesh
s409                    Chhota Bheem & Krishna: Mayanagari
s410     Chhota Bheem & Krishna: Pataliputra- City of t...
s412                Chhota Bheem And The Crown of Valhalla
s411                    Chhota Bheem And The Broken Amulet
s416                              Chhota Bheem aur Krishna
s413                  Chhota Bheem and the Incan Adventure
s414                   Chhota Bheem and The ShiNobi Secret
s415                              Chhota Bheem Aur Hanuman
s417                   Chhota Bheem aur Krishna vs Zimbara
s421                  Chhota Bheem: Dholakpur to Kathmandu
s423                        Chhota Bheem: Journey to Petra
s424                       Chhota Bheem: Master of Shaolin
s425                     Chhota Bheem: The Rise of Kirmada
s2718                Chhota Bheem and the Curse 

No doubt about it. So I guess our dataset is cleaned to the highest achievable degree. So let's write it to new .csv file

In [23]:
with open('netflix_clean.csv', 'wb') as file:
    netflix.to_csv(file)